In [1]:
import os
import pandas as pd
# from nltk import ngrams, FreqDist

In [2]:
import glob
import errno

file_path = './temp/train.txt'

In [3]:
import re
def read_files(file_name): 
    lines = open(file_name).read().split("\n")
    
    return lines
    

In [4]:
words_corpus_sentence=read_files(file_path)

In [5]:
print(len(words_corpus_sentence))

456867


In [6]:

grammar_bigram={}


i=1

prev_word=""
current_word=""
next_word=""


while i<len(words_corpus_sentence)-1:
    
    prev_word=words_corpus_sentence[i-1]
    current_word=words_corpus_sentence[i]
    next_word=words_corpus_sentence[i+1]
    
    
    
    gtype=current_word
    
    prev_word=prev_word.split("   ")[0]
    current_word=current_word.split("   ")[0]
    next_word=next_word.split("   ")[0]
    
    type_error="correct"
    
    
    gtype=gtype.split("   ")
      
    type_error=="correct"
        
        
    if len(gtype)>=3:
#         print(gtype[2])
        type_error=gtype[2]
        
#     print(type_error)

    if type_error=="saving":
        i+=1
        continue
        
    
    if type_error not in grammar_bigram:        
        grammar_bigram[type_error]={}
        
        

    temp1=(prev_word,current_word)
    temp2=(current_word,next_word)


    if temp1 not in grammar_bigram[type_error]:
        grammar_bigram[type_error][temp1]=0


    if temp2 not in grammar_bigram[type_error]:
        grammar_bigram[type_error][temp2]=0

   


    grammar_bigram[type_error][temp1]+=1        
    grammar_bigram[type_error][temp2]+=1
    
    
    i+=1
    
    
    

In [7]:
print (len(grammar_bigram["Nn"]))

# print (grammar_bigram["correct"])



4618


In [8]:
grammar_bigram = pd.DataFrame(grammar_bigram)
# grammar_bigram

In [9]:
sum_counts = grammar_bigram.sum(axis = 1).sort_values(ascending = False)

In [10]:
grammar_bigram = grammar_bigram.reindex(sum_counts.index)

In [11]:
print(grammar_bigram.sum())

ArtOrDet     10400.0
Cit           3154.0
Mec           5372.0
Nn            6108.0
Npos           500.0
Others        3214.0
Pform          346.0
Pref          1728.0
Prep          3598.0
Rloc-        15504.0
SVA           2410.0
Sfrag         1102.0
Smod           416.0
Spar          1398.0
Srun          2576.0
Ssub          1020.0
Trans         3134.0
Um           15726.0
V0             700.0
Vform         2518.0
Vm             902.0
Vt            5948.0
WOadv         1462.0
WOinc         5784.0
Wa              96.0
Wci          12906.0
Wform         3432.0
Wtone         2926.0
correct     799348.0
dtype: float64


In [12]:
# Laplace Smoothing
grammar_bigram = grammar_bigram + 1
grammar_bigram.sum()

ArtOrDet     14437.0
Cit           4645.0
Mec           8756.0
Nn           10726.0
Npos           855.0
Others        5230.0
Pform          618.0
Pref          2881.0
Prep          6003.0
Rloc-        23372.0
SVA           4347.0
Sfrag         1720.0
Smod           658.0
Spar          2308.0
Srun          3973.0
Ssub          1689.0
Trans         4872.0
Um           22229.0
V0            1171.0
Vform         4540.0
Vm            1516.0
Vt           10137.0
WOadv         2398.0
WOinc         8618.0
Wa             179.0
Wci          21126.0
Wform         6368.0
Wtone         4357.0
correct     927113.0
dtype: float64

In [13]:
# Convert to Probabilites
grammar_bigram_probs = grammar_bigram/grammar_bigram.sum()
# all_unigrams_probs

In [14]:
import math
grammar_bigram_log = grammar_bigram_probs.applymap(math.log)

In [15]:
def sentence_prob(sentence):
    wordprobs = grammar_bigram_log.loc[sentence] 
    #print(wordprobs)
    return wordprobs.sum()


In [16]:
sentences = ["environment","friendly"]
sprobs = pd.DataFrame(sentence_prob(sentences))

sprobs
# sprobs.transpose().iplot()

,0
ArtOrDet,0.000000
Cit,0.000000
Mec,-49.495153
Nn,-76.880044
Npos,0.000000
Others,0.000000
Pform,0.000000
Pref,0.000000
Prep,0.000000
Rloc-,-83.889856


In [17]:
cprobs = grammar_bigram_log.sum()
cprobs = cprobs / cprobs.sum()
cprobs = cprobs.apply(math.log)

In [18]:
def nbestimate(sentence):
    probs = grammar_bigram_log.loc[sentence].sum()
    probs = probs + cprobs
    return probs

In [19]:
sprobs = pd.DataFrame(nbestimate(sentences))

print(sprobs)


                    0
ArtOrDet    -4.106826
Cit         -5.257440
Mec        -53.827099
Nn         -80.870979
Npos        -6.916137
Others      -4.917146
Pform       -7.232727
Pref        -5.552843
Prep        -4.715229
Rloc-      -87.277730
SVA        -19.931483
Sfrag       -6.275235
Smod        -7.372052
Spar       -19.928698
Srun       -27.300961
Ssub       -19.658544
Trans      -28.068191
Um         -47.580075
V0          -6.581517
Vform       -4.921355
Vm          -6.277550
Vt         -21.159005
WOadv      -12.886993
WOinc      -12.904818
Wa          -8.657904
Wci        -76.655493
Wform      -60.152209
Wtone      -12.989627
correct  -1292.553053


In [20]:
s=sprobs.idxmax()

print (s)
print(sprobs.max())

0    ArtOrDet
dtype: object
0   -4.106826
dtype: float64


In [21]:
file_path = './temp/dev.txt'

In [22]:
sentence_test=read_files(file_path)

In [23]:
f = open("result.txt", "a")

In [24]:
def check_error(c_sentence):
#     print(c_sentence)
    
#     print()
    
    sprobs = pd.DataFrame(nbestimate(c_sentence[0:2]))
    
#     print(sprobs.max())
#     print(sprobs.idxmax())
    
    max_v=float(sprobs.max())
    type_e=list(sprobs.idxmax())[0]
    word=[]
    
    
#     print(float(max_v))
#     print(list(sprobs.idxmax())[0])
    
    
    i=2
    while i<len(c_sentence)-1:        
#         print(c_sentence[i:i+2])
        
        
        sprobs = pd.DataFrame(nbestimate(c_sentence[i:i+2]))
        
        
        if max_v<float(sprobs.max()):
            max_v=float(sprobs.max())
            type_e=list(sprobs.idxmax())[0]
            word=c_sentence[i:i+2]
        
            
        i+=1
            

        
    
    
#     print(max_v)
#     print(type_e)
#     print(word[0])
    
    return word[0],type_e
    


In [25]:
def write_to_file(sentence_list,found_word,found_grammar):
    
    f = open("result.txt", "a")
    for temp in sentence_list:
#         print(temp)
        
        if temp==found_word:
            
            print(temp+"   "+found_grammar)
            f.write(temp+"   "+found_grammar)
            f.write("\n")
            continue
        f.write(temp)
        f.write("\n")
        
    


In [26]:
i=0

temp=[]

while i <len(sentence_test):
#     print(sentence_test[i])
    
    
    
    
    if sentence_test[i]=='.':
        found_word,found_grammar=check_error(temp)   
        
        print(found_word,found_grammar)
        write_to_file(temp,found_word,found_grammar)
        
        
        temp[:] = []
        i+=1
        continue
        
        
        
        
    else:
        temp.append(sentence_test[i])
     
        
        
        
    i+=1



better Prep
better   Prep
stay Wci
stay   Wci
baby Rloc-
baby   Rloc-
traditional Rloc-
traditional   Rloc-
uses Rloc-
uses   Rloc-
enough Vt
enough   Vt
showing Wci
showing   Wci
mission Wci
mission   Wci
all SVA
all   SVA
our Prep
our   Prep
our   Prep
may Prep
may   Prep
few Vt
few   Vt
only Cit
only   Cit
jobs Nn
jobs   Nn
their SVA
their   SVA
parties Nn
parties   Nn
been Nn
been   Nn
compulsory Wci
compulsory   Wci
permanently Wci
permanently   Wci
comparing Um
comparing   Um
insurance Rloc-
insurance   Rloc-
determines Nn
determines   Nn
ensure Prep
ensure   Prep
sound Wci
sound   Wci
economic Vform
economic   Vform
well Prep
well   Prep
encouraging Nn
encouraging   Nn
encouraging   Nn
making Wform
making   Wform
often Nn
often   Nn
and Npos
and   Npos
house Rloc-
house   Rloc-
contribute Nn
contribute   Nn
elderly Pform
elderly   Pform
engineering Trans
engineering   Trans
oil Rloc-
oil   Rloc-
minimum Um
minimum   Um
including Wci
including   Wci
molten Wci
molten   Wci
over N

IndexError: list index out of range

In [ ]:
f.close()